In [22]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import odeint
import plotly.graph_objects as go
def funciones_refuerzos(t):
    r_a = 34000 * np.cos(99*t)  # Example function for reinforcements arriving for Force A
    r_b = 35000* np.sin(58*t) # Example function for reinforcements arriving for Force B
    return r_a, r_b

def batalla_con_refuerzos(y, t, alpha=20, beta=15, r_func=None):
    a, b = y
    if r_func:
        r_a, r_b = r_func(t)
        da_dt = -alpha * b + r_a
        db_dt = -beta * a + r_b
    else:
        da_dt = -alpha * b
        db_dt = -beta * a
    return [da_dt, db_dt]
# Function to find the end time when either force reaches 0
def encontrar_tiempo_final(alpha, beta, a0, b0, r_func, t_max):
    t = np.linspace(0, t_max, 1000)
    y0 = [a0, b0]
    solution = odeint(batalla_con_refuerzos, y0, t, args=(alpha, beta, r_func))
    a_values, b_values = solution.T
    end_time = None
    for i in range(len(t)):
        if a_values[i] <= 0 or b_values[i] <= 0:
            end_time = t[i]
            break
    return end_time

# Function to plot the simulation from time 0 to end time
def graficar_simulacion_refuerzos(alpha, beta, a0, b0, r_func, t_max):
    end_time = encontrar_tiempo_final(alpha, beta, a0, b0, r_func, t_max)
    if end_time is None:
        print("Both forces survive the battle.")
        return

    t = np.linspace(0, end_time, 1000)
    y0 = [a0, b0]
    solution = odeint(batalla_con_refuerzos, y0, t, args=(alpha, beta, r_func))
    a_values, b_values = solution.T

    # Create a Plotly figure
    fig = go.Figure()

    fig.add_trace(go.Scatter(x=t, y=a_values, mode='lines', name='Force A'))
    fig.add_trace(go.Scatter(x=t, y=b_values, mode='lines', name='Force B'))

    # Update the layout
    fig.update_layout(
        xaxis_title='Time',
        yaxis_title='Strength',
        title='Battle Simulation with Reinforcements',
    )

    # Show the plot
    fig.show()

    # Print the time when either force reaches 0
    print(f"Either Force A or Force B reaches 0 at time t = {end_time:.24f}")

# Parameters
alpha = 20  # Rate of attrition for Force A
beta = 15   # Rate of attrition for Force B
a0 = 1000   # Initial strength of Force A
b0 = 800    # Initial strength of Force B
t_max = 32

# Plot the simulation with reinforcements
graficar_simulacion_refuerzos(alpha, beta, a0, b0, funciones_refuerzos, t_max)

Either Force A or Force B reaches 0 at time t = 0.064064064064064063508397


In [23]:
import numpy as np
import plotly.graph_objs as go
from scipy.integrate import odeint

def retrato_refuerzos(uinits,r_func, max_time=20):
    # Time values
    t_values = np.linspace(0, max_time, 1000000)

    # Create a Plotly figure
    fig = go.Figure()

    # Solve the system and add traces for each set of initial conditions
    for uinit in uinits:
        # odeint(batalla_con_refuerzos, y0, t, args=(alpha, beta, r_func))
        u = odeint(batalla_con_refuerzos, uinit, t_values,args=(alpha, beta, r_func))
        a_values = u[:, 0]
        b_values = u[:, 1]
        for i in range(len(t_values)):
            if a_values[i] <= 0 or b_values[i] <= 0:
                tiempo_final = t_values[i]
                a_values = a_values[0:i + 1]
                b_values = b_values[0:i + 1]
                break
        fig.add_trace(go.Scatter(x=a_values, y=b_values, mode='lines', name=str(uinit)))

    # Update the layout to set the same scale for both axes
    fig.update_layout(
        xaxis=dict(scaleanchor="y", scaleratio=1),
        yaxis=dict(scaleanchor="x", scaleratio=1),
        xaxis_title='Strength of Force A',
        yaxis_title='Strength of Force B',
        title='Fuego Directo Phase Portrait with Multiple Initial Conditions',
        legend_title='Initial Conditions'
    )

    # Show the plot
    fig.show()

# Initial conditions
uinits = [[1000, 800], [2000, 1900], [8000, 6000],[6000,5000],[3000,4000]]

# Call the function to plot the phase portrait
retrato_refuerzos(uinits,funciones_refuerzos)